In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example,here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing,CSV file I/O (e.g. pd.read_csv)
import re
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(44) # better for debugging to have seed 
import time
import datetime

# Input data files are available in the read-only "../input/" directory
# For example,running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname,_,filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname,filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/,but they won't be saved outside of the current session

/kaggle/input/arabic-poetry/arabic-poems.txt
/kaggle/input/tiny-arabic-poetry/small-arabic-poems.txt


In [2]:
with open('../input/tiny-arabic-poetry/small-arabic-poems.txt','r',encoding='utf-8') as f:
    poems=f.read()

In [3]:
print("Number of characters in dataset=",len(poems))

Number of characters in dataset= 10000016


In [4]:
poems[:1000]

'بدت تختال في حلل الجمال\nوجادت بالزيارة والوصال\nتميس فلا يعادلها قضيب\nوإن ترنو تداعب بالنصال\nبمبسمها لعمر أبيك در\nوفي أعماقها نبع الزلال\nوخصر يستبيك إذا تولت\nكغصن البان في كثب الرمال\nتبدت كالقضيب على كثيب\nوجلت كالمنيرة في الليالي\nفقمت أداعب الوجنات منها\nوألثم ثغرها حكي اللآلي\nوأهصر غصنها ضما ولما\nوألهو باليمين وبالشمال\nومن خلق العفاف لنا رقيب\nبطهر الحب في حسن الخلال\nوقد غاب الرقيب وطاب أنسي\nوطير الحب يصدح بامتثال\nتقول أراك تظهر لي اشتياقا\nوتفعل كالمودع للرحال\nفقلت لها رويدك إن قلبي\nوحبي لم يكن يوما بسالي\nولكني عزمت وفي عزم\nليهزأ بالأسنة والعوالي\nسئمت من المقام وكل شيء\nإذا ما دام يسأم لا محال\nسأضرب في الحياة بسهم جدي\nونحظى بالمسرة والوصال\nوأرجع إن يشا الباري قريبا\nإليك لنبتني صرح المعالي\nفرقرق لؤلؤ في مقلتيها\nوصاحت آه من مر الليالي\nأتترك يا حبيب الروح قلبا\nيكاد يذوب من شوق لحالي\nوتسلوني وأنت نعيم روحي\nولا ترعى المودة أو تبالي\nليحفظ الإله بكل أرض\nويرزقك السلامة في الكمال\nوجمنا لم نحر قولا ولكن\nنهير الدمع فاض على التوالي\nولاح الصبح من تحت الثريا\nكم

In [5]:
# gather all unique characters within dataset
chars=sorted(list(set(poems)))
vocab_size=len(chars)
print(''.join(chars))
print(vocab_size)

	
 ؟ءآأؤإئابةتثجحخدذرزسشصضطظعغـفقكلمنهوىيپچ  
45


In [6]:
# tokenize chars by mapping to integers
stoi={ch:i for i,ch in enumerate(chars)}
itos={i:ch for i,ch in enumerate(chars)}
encode=lambda s:[stoi[c] for c in s] # string --> list of integers
decode=lambda l:''.join([itos[i] for i in l]) # list of integers --> string

In [7]:
import torch
# encode dataset
data=torch.tensor(encode(poems),dtype=torch.long)
print(data.shape,data.dtype)
print(data[:1000])

torch.Size([10000016]) torch.int64
tensor([11, 18, 13,  2, 13, 17, 13, 10, 34,  2, 31, 40,  2, 16, 34, 34,  2, 10,
        34, 15, 35, 10, 34,  1, 38, 15, 10, 18, 13,  2, 11, 10, 34, 21, 40, 10,
        20, 12,  2, 38, 10, 34, 38, 24, 10, 34,  1, 13, 35, 40, 22,  2, 31, 34,
        10,  2, 40, 28, 10, 18, 34, 37, 10,  2, 32, 25, 40, 11,  1, 38,  8, 36,
         2, 13, 20, 36, 38,  2, 13, 18, 10, 28, 11,  2, 11, 10, 34, 36, 24, 10,
        34,  1, 11, 35, 11, 22, 35, 37, 10,  2, 34, 28, 35, 20,  2,  6, 11, 40,
        33,  2, 18, 20,  1, 38, 31, 40,  2,  6, 28, 35, 10, 32, 37, 10,  2, 36,
        11, 28,  2, 10, 34, 21, 34, 10, 34,  1, 38, 17, 24, 20,  2, 40, 22, 13,
        11, 40, 33,  2,  8, 19, 10,  2, 13, 38, 34, 13,  1, 33, 29, 24, 36,  2,
        10, 34, 11, 10, 36,  2, 31, 40,  2, 33, 14, 11,  2, 10, 34, 20, 35, 10,
        34,  1, 13, 11, 18, 13,  2, 33, 10, 34, 32, 25, 40, 11,  2, 28, 34, 39,
         2, 33, 14, 40, 11,  1, 38, 15, 34, 13,  2, 33, 10, 34, 35, 36, 40, 20,
     

In [8]:
# train/val split=90/10
n=int(0.9*len(data))
train_data=data[:n]
val_data=data[n:]

In [9]:
# hparams
device='cuda' if torch.cuda.is_available() else 'cpu'
print(device)
batch_size=64
block_size=256
max_iters=10000
eval_interval=500
lr=3e-4
eval_iters=200
n_embd=384
n_head=6
n_layer=6
dropout=0.2

cuda


In [10]:
# separate data into batches

def get_batch(split):
    data=train_data if split == 'train' else val_data
    ix=torch.randint(len(data)-block_size,(batch_size,))
    x=torch.stack([data[i:i+block_size] for i in ix]) 
    y=torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y=x.to(device),y.to(device)
    return x,y

In [11]:
@torch.no_grad()
def estimate_loss():
    out={}
    model.eval()
    for split in ['train','val']:
        losses=torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y=get_batch(split)
            logits,loss=model(X,Y)
            losses[k]=loss.item()
        out[split]=losses.mean()
    model.train()
    return out

In [12]:
class Head(nn.Module):
    # single head of self-attention
    def __init__(self,head_size):
        super().__init__()
        self.key=nn.Linear(n_embd,head_size,bias=False)
        self.query=nn.Linear(n_embd,head_size,bias=False)
        self.value=nn.Linear(n_embd,head_size,bias=False)
        self.register_buffer('tril',torch.tril(torch.ones(block_size,block_size)))
        self.dropout=nn.Dropout(dropout)
    
    def forward(self,x):
        B,T,C=x.shape
        k=self.key(x) # (B,T,C)
        q=self.query(x) # (B,T,C)
        wei=q @ k.transpose(-2,-1) * C**-0.5 # (B,T,C) @ (B,C,T) ---> (B,T,T) 
        wei=wei.masked_fill(self.tril[:T,:T]==0,float('-inf')) # (B,T,T)
        wei=F.softmax(wei,dim=-1) # (B,T,T)
        wei=self.dropout(wei)
        v=self.value(x)
        out=wei @ v # (B,T,T) @ (B,T,C) ---> (B,T,C) 

        return out

In [13]:
class MultiHeadAttention(nn.Module):
    def __init__(self,num_heads,head_size):
        super().__init__()
        self.heads=nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj=nn.Linear(num_heads * head_size,n_embd)
        self.dropout=nn.Dropout(dropout)
        
    def forward(self,x):
        out=torch.cat([h(x) for h in self.heads],dim=-1)
        out=self.dropout(self.proj(out))
        
        return out

In [14]:
class FeedForward(nn.Module):
    def __init__(self,n_embd):
        super().__init__()
        self.net=nn.Sequential(
            nn.Linear(n_embd,4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd,n_embd),
            nn.Dropout(dropout),
        )
        
        
    def forward(self,x):
        return self.net(x)

In [15]:
class Block(nn.Module):
#     Tranformer block: communication (self-attention) followed by computation (feed-forward)

    def __init__(self,n_embd,n_head):
        super().__init__()
        head_size=n_embd // n_head
        self.sa=MultiHeadAttention(n_head,head_size)
        self.ff=FeedForward(n_embd)
        self.ln1=nn.LayerNorm(n_embd)
        self.ln2=nn.LayerNorm(n_embd)
        
    def forward(self,x):
        x=x + self.sa(self.ln1(x))
        x=x + self.ff(self.ln2(x))
        return x

In [16]:
class DiwanGPT(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.token_embedding=nn.Embedding(vocab_size,n_embd)
        self.pos_embedding=nn.Embedding(block_size,n_embd)
        self.blocks=nn.Sequential(*[Block(n_embd,n_head) for _ in range(n_layer)])
        self.ln_f=nn.LayerNorm(n_embd) # final layer norm
        self.lm_head=nn.Linear(n_embd,vocab_size)
        self.apply(self._init_weights)
    
    def _init_weights(self,module):
        if isinstance(module,nn.Linear):
            torch.nn.init.normal_(module.weight,mean=0.0,std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module,nn.Embedding):
            torch.nn.init.normal_(module.weight,mean=0.0,std=0.02)
            
    def forward(self,idx,targets=None):
        B,T=idx.shape
        tok_emb=self.token_embedding(idx) # (B,T,C)
        pos_emb=self.pos_embedding(torch.arange(T,device=device)) # (T,C)
        # (B,T,C)
        x=tok_emb + pos_emb 
        x=self.blocks(x)
        x=self.ln_f(x)
        logits=self.lm_head(x) # (B,T,vocab_size)
        if targets is None:
            loss=None
        else:
            B,T,C=logits.shape
            logits=logits.view(B*T,C)
            targets=targets.view(B*T)
            loss=F.cross_entropy(logits,targets)
        return logits,loss
    
    def generate(self,idx,max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond=idx[:,-block_size:]
            logits,loss=self(idx_cond)
            logits=logits[:,-1,:] # (B,C)
            probs=F.softmax(logits,dim=-1)
#             sample from distribution using probs
            idx_next=torch.multinomial(probs,num_samples=1) # (B,1)
            idx=torch.cat((idx,idx_next),dim=1) # (B,T + 1)
        return idx

In [17]:
model=DiwanGPT()
m=model.to(device)
print(sum(p.numel() for p in m.parameters())/1e6,'M parameters') # num of params in model

10.773549 M parameters


In [18]:
optimizer=torch.optim.AdamW(model.parameters(),lr=lr)

In [19]:
# train
start_time=time.time()
for i in range(max_iters):
    if i % eval_interval == 0 or iter == max_iters - 1:
        losses=estimate_loss()
        elapsed_time=time.time() - start_time
        print(f"Time elapsed: {str(datetime.timedelta(seconds=int(elapsed_time)))} step {i}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    xb,yb=get_batch('train')
    logits,loss=model(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

Time elapsed: 0:00:32 step 0: train loss 3.8994, val loss 3.8971
Time elapsed: 0:03:26 step 500: train loss 2.4514, val loss 2.4228
Time elapsed: 0:06:19 step 1000: train loss 2.1692, val loss 2.1329
Time elapsed: 0:09:12 step 1500: train loss 2.0350, val loss 2.0049
Time elapsed: 0:12:06 step 2000: train loss 1.9619, val loss 1.9421
Time elapsed: 0:14:59 step 2500: train loss 1.9158, val loss 1.8983
Time elapsed: 0:17:52 step 3000: train loss 1.8819, val loss 1.8681
Time elapsed: 0:20:46 step 3500: train loss 1.8564, val loss 1.8462
Time elapsed: 0:23:39 step 4000: train loss 1.8346, val loss 1.8293
Time elapsed: 0:26:32 step 4500: train loss 1.8190, val loss 1.8134
Time elapsed: 0:29:26 step 5000: train loss 1.8043, val loss 1.7995
Time elapsed: 0:32:19 step 5500: train loss 1.7918, val loss 1.7866
Time elapsed: 0:35:12 step 6000: train loss 1.7782, val loss 1.7782
Time elapsed: 0:38:05 step 6500: train loss 1.7682, val loss 1.7687
Time elapsed: 0:40:59 step 7000: train loss 1.7533, 

In [20]:
# generate
context=torch.zeros((1,1),dtype=torch.long,device=device)
print(decode(m.generate(context,max_new_tokens=1000)[0].tolist()))
#open('/kaggle/working/generated_poetry.txt','w').write(decode(m.generate(context,max_new_tokens=10000)[0].tolist()))

	انينا  سيدا
في تمرة   وبما نابى
طير موزنا والشيوخ صادية
 فبكيس 
وكان ايقظواد 
ووصف
كان للبرق أمنهما
تعتاد بهم  فتى
هذا الطيور  دون أرتحال صمت النبوءة
هذه العقار ولن تعقلون لغضبا
إلى الصيالحون
أعطيهما  لصدوريه
طليق أمريك  حيثما تجوز باعتراضاتي كخطآطائك  خطى الطفلة
تحت الغبار
مكشوب  يا موتي انضجاع 
تافها  غزلتي  نضحت تموتي  وغبي
الشواقي  خذي
خيم الزراعي  لوعة سرقت تسعفي
أمليكة الظلام  ظل القمر 
كي يدفن بالزيتون
فترمي كامله  قبي
عشرين  موت عشرين  والخطر الغريزة
حتى يقيسون الزيتون
ويسقط ويلقيه
ويدعي أجبتين فمك
يا كعباس القمر أبحث عني
أقول كاتب عن فراقها ؟
ونحبتين  
وقد يبكي غربتي
إطاروا الصخر من أين دقتنا
وأجبتين قرما تقرأ
بكل التقاطع  لكي يخيب
ودمعي سنة التراب يربو
قد فعلت معرة فأتى
وحملت بجلجال قمري
واستقبلته دمعي 
وأتعبتي 
منشأة حملت أعرضنا
فبعلمي 
أعيم نهرا من هناك؟
وسمي 
القلوب تصوير النار
وضغطت ركوة الخاصم 
إذن للجلابديو 
أهمي زهرة زهر الغاوي
وما السمع المروي 
أولادي  
وألف أهمي وأدري 
مشيرا بالخضم 
وجدا تمادلا عليه 
طيبي 
يا فمي  يا يسرب الغاضة الضرامي
رعشت يدي 
وأين القرى كهرب 
تن